# Part of speech grammer parser
Use template to grab words that fit with the part of speech grammar pattern.  
Examples of the format of the grammar parser are:  
* `<DT><NN><VB><NN>`
 * Returns `<DT><NN><VBZ><NN>`
 * e.g. The cat plays piano
* `!<DT><NN><VB><NN>`
 * Returns `<NN><VBZ><NN>`
 * e.g. cat plays piano
* `<DT><NN>*<NN>`
 * Returns `<DT><NN><VBZ><NN>`
 * e.g. The cat plays piano
 

In [1]:
import nltk
# sentence = "The cat plays piano."

with open("ITIL Books/ITIL 3/Service operation chapter 4/Service operation chapter 4 - 4.txt") as file:
    file_contents = file.read()
    
sentences = nltk.tokenize.sent_tokenize(file_contents)

sentence = "Event Management is the process that monitors all events that occur through the IT infrastructure to allow for normal operation and also to detect and escalate exception conditions."

ontology_main = {}

for sentence in sentences:
    ontology = {}
    
    tokens = nltk.word_tokenize(sentence)
    
    ### Part of speech tagging ###
    part_of_speech_array = nltk.pos_tag(tokens)
    # print(part_of_speech_array)
    # print("-" * 100)
    
    # def search_pos_sentence_with_grammar(grammar, sentence, ordering):
    #     tags_to_search_for = []
    #     for all_pos_tags in grammar:
    #         all_pos_tags = all_pos_tags.split("/")
    #         # print(all_pos_tags)
    #         tags_to_search_for.append(all_pos_tags)
    #         
    #     print(tags_to_search_for)
    #     
    #     phrase = []
    #     found_start_word = False
    #     tag_count = 0
    #     word_count = 0
    #     for pos_word in sentence:
    #         if tag_count < len(tags_to_search_for) and pos_word[1] in tags_to_search_for[tag_count]:
    #             if ordering is True:
    #                 
    #             tag_count = tag_count + 1
    #             phrase.append(pos_word)
    #         word_count = word_count + 1
    #     print(phrase)
    #     return phrase
    #         
    #         
    #     
    #         
    # def pos_grammar_parse(grammar, sentence, ordering=False):
    #     grammar = [item.replace("<", "").upper() for item in grammar.split(">") if item is not '']
    #     # grammar = [item.split("/") for item in grammar]
    #     # print(grammar)
    #     return search_pos_sentence_with_grammar(grammar, sentence, ordering)
    #     # print(grammar)
    
    #%%
    from nltk.stem import WordNetLemmatizer 
    lemmatizer = WordNetLemmatizer()
    
    part_of_speech_array_lemmatized = []
    
    for part_of_speech in part_of_speech_array:
        part_of_speech_array_lemmatized.append(
            (lemmatizer.lemmatize(part_of_speech[0]), part_of_speech[1])
        ) 
    
    # print(part_of_speech_array_lemmatized)
    
    # print(part_of_speech_array)
    
    #%%
    from nltk.corpus import stopwords
    stop_words = stopwords.words('english')
    
    part_of_speech_array_lemmatized_no_stopwords = [word_pos for word_pos in part_of_speech_array_lemmatized if not word_pos[0] in stop_words]
    
    # print(part_of_speech_array_lemmatized_no_stopwords)
    
    #%%
    # noun_phrase_grammar = "NP: {<DT>?<JJ>*<NN|NNP|NNS>*}"
    # noun_phrase_grammar = "NP: {<JJ>*<NN|NNP|NNS>*}"
    noun_phrase_grammar = "NP: {<JJ>*<NN|NNP|NNS>+}"
    # print(pos_grammar_parse(grammar, part_of_speech_array))
    
    regex = nltk.RegexpParser(noun_phrase_grammar)
    noun_phrases_chunked = regex.parse(part_of_speech_array_lemmatized)
    noun_phrases = [item for item in noun_phrases_chunked.subtrees() if item.label() == "NP"]
    
    # print(noun_phrases)
    
    # for item in noun_phrases_chunked:
    #     print(item)
    # print()
    # 
    # for concept in noun_phrases:
    #     print(concept)
    #     
    # print()
    
    # relation_grammer = """
    #     Relation: {<NP>?.*?<VBZ|VBP|IN|VB>+.*?<NP>?}
    #     """
    
    relation_grammer = """
        Relation: {.*?<VBZ|VBP|IN|VB>+.*?}
        """
    regex = nltk.RegexpParser(relation_grammer)
    parser = regex.parse(noun_phrases_chunked)
    # relation_phrases = [item for item in parser.subtrees() if item.label() == "Relation"]
    # 
    # # print(relation_phrases)
    # 
    # # for parse in parser:
    # #     print(parse)
    # 
    # relation_new = []
    # # TODO: Find a less hacky way to do this
    # for index, relation in enumerate(relation_phrases):
    #     # print(relation)
    # 
    #     temp_relation = []
    #     if index is not 0:
    #         temp_relation.append(relation_phrases)
    #         temp_relation.append(item for item in relation)
    #         # print(f"TEMP RELATION: {temp_relation}")
    #         relation = temp_relation
    # 
    #     # print(temp_relation)
    #     relation_new.append(relation)
    #     # print(relation_new)
    # 
    # print()
    # 
    # # for relation in relation_new:
    # #     for item in relation:
    # #         print(item)
    #     
    # for relation in relation_phrases:
    #     print(relation)
    
    # for element in parser.subtrees():
    #     if element.label() in ['NP', 'Relation']:
    #         print(element)
            
    # Chunk conjunctives
    conjunctive_grammer = "Conjunctive: {.*?<CC>+.*?}"
    regex = nltk.RegexpParser(conjunctive_grammer)
    parser = regex.parse(parser)
    
    # print(parser)
    
    # for element in parser.subtrees():
    #     if element.label() in ['NP', 'Relation', 'Conjunctive']:
    #         print(element)
    
    relations = [element for element in parser.subtrees() if element.label() in ['NP', 'Relation', 'Conjunctive']]
    
    noun_phrase_indexes = [index for index, element in enumerate(relations) if element.label() == "NP"]
    
    # print(noun_phrase_indexes)
    
    relationships = []
    for index, element in enumerate(noun_phrase_indexes[::2]):
        relationships.append(relations[noun_phrase_indexes[index]:noun_phrase_indexes[index + 1] + 1:])
    
    # ontology = {}
        
    # print(sentence)    
    
    for relationship in relationships:
        conjunctive_relation = []
        temp_relation = []
        
        first_concept = " ".join([i[0] for i in relationship[0].leaves()])
        
        # print(first_concept.replace("¦ ", ""))
        
        # TODO: Make this work better make conjuctive relation addative
        # In other words make conjunctive into [[]] with words added to [["is", "here"]]
        # then use conjunctive to add new value so [["is", "here"], ["is", "an", "animal"]]
        for element in relationship[1::]:
            if element.label() == "Relation":
                temp_relation.append(" ".join(i[0] for i in element.leaves()))
                for conjunctive_list in conjunctive_relation[::]:
                    conjunctive_relation[-1].append(" ".join(i[0] for i in element.leaves()))
            elif element.label() == "Conjunctive":
                conjunctive_relation.append(temp_relation)
                conjunctive_relation.append(temp_relation[:-1:])
                
                # for conjuctive_list in conjunctive_relation:
                # temp_relation.append(" ".join(i[0] for i in element.leaves()))
                # conjunctive_relation.append(temp_relation)
                # conjunctive_relation.append(temp_relation[:-1:].append(" ".join(i[0] for i in element.leaves())))
                
            elif element.label() == "NP":
                temp_relation.append("::" + " ".join(i[0] for i in element.leaves()))
                for conjuctive_list in conjunctive_relation:
                    conjunctive_list.append("::" + " ".join(i[0] for i in element.leaves()))
            # print(element)
            
            # print(element)
        # print(temp_relation)
        print(f"conjunctive_relation: {conjunctive_relation}")

        print("-" * 100)
            
        # first_concept = " ".join([i[0] for i in relationship[0].leaves()])
        # 
        # rel = " ".join(" ".join([j[0] for j in i.leaves()]) for i in relationship[1::] if i.label() != "NP")
        # rel = f"{rel}::{' '.join([i[0] for i in relationship[-1].leaves()])}"
        # 
        # if first_concept not in ontology.keys():
        #     ontology[first_concept] = []
        # ontology[first_concept].append(rel)
        
        print(f"ontology: {ontology}")
    
        for element, value in ontology:
            ontology_main[element] = ontology_main[element].append[value.flatten]
        
print(f"ontology_main: {ontology_main}")


conjunctive_relation: []
----------------------------------------------------------------------------------------------------
ontology: {}
conjunctive_relation: []
----------------------------------------------------------------------------------------------------
ontology: {}
conjunctive_relation: []
----------------------------------------------------------------------------------------------------
ontology: {}
conjunctive_relation: []
----------------------------------------------------------------------------------------------------
ontology: {}
conjunctive_relation: [['is', 'of', '::process'], ['of', 'of', '::process', '::process']]
----------------------------------------------------------------------------------------------------
ontology: {}
conjunctive_relation: []
----------------------------------------------------------------------------------------------------
ontology: {}
conjunctive_relation: []
----------------------------------------------------------------------------

In [14]:
# Methods fot non conjunctive relation selections (ises code from iteration_2_part_of_speech_grammar_parser)
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
# sentence = "The cat plays piano."
sentence = "Event Management is the process that monitors all events that occur through the IT infrastructure to allow for normal operation and also to detect and escalate exception conditions."

def tokenize_sentence(sentence):
    tokens = nltk.word_tokenize(sentence)
    return nltk.pos_tag(tokens)
    
def lemmatize_tokens(part_of_speech_array):
    lemmatizer = WordNetLemmatizer()
    
    part_of_speech_array_lemmatized = []
    
    for part_of_speech in part_of_speech_array:
        part_of_speech_array_lemmatized.append(
            (lemmatizer.lemmatize(part_of_speech[0]), part_of_speech[1])
        )
    return part_of_speech_array_lemmatized

def remove_stopwords_from_tokens(part_of_speech_array_lemmatized):
    stop_words = stopwords.words('english')
    
    part_of_speech_array_lemmatized_no_stopwords = [word_pos for word_pos in part_of_speech_array_lemmatized if not word_pos[0] in stop_words]
    
    return part_of_speech_array_lemmatized_no_stopwords

def get_noun_phrases(part_of_speech_array_lemmatized):
    noun_phrase_grammar = "NP: {<JJ>*<NN|NNP|NNS>+}"
    
    regex = nltk.RegexpParser(noun_phrase_grammar)
    noun_phrases_chunked = regex.parse(part_of_speech_array_lemmatized)
    noun_phrases = [item for item in noun_phrases_chunked.subtrees() if item.label() == "NP"]
    
    return noun_phrases_chunked, noun_phrases

def get_relations(noun_phrases_chunked):
    relation_grammer = """
    Relation: {.*?<VBZ|VBP|IN|VB>+.*?}
    """
    regex = nltk.RegexpParser(relation_grammer)
    parser = regex.parse(noun_phrases_chunked)
            
    # Chunk conjunctives
    conjunctive_grammer = "Conjunctive: {.*?<CC>+.*?}"
    regex = nltk.RegexpParser(conjunctive_grammer)
    parser = regex.parse(parser)
    
    relations = [element for element in parser.subtrees() if element.label() in ['NP', 'Relation', 'Conjunctive']]
    
    noun_phrase_indexes = [index for index, element in enumerate(relations) if element.label() == "NP"]
        
    relationships = []
    for index, element in enumerate(noun_phrase_indexes[::2]):
        relationships.append(relations[noun_phrase_indexes[index]:noun_phrase_indexes[index + 1] + 1:])
        
    return relationships


def create_ontology(relationships):    
    ontology = {}
        
    for relationship in relationships:
        temp_relation = []
        
        first_concept = " ".join([i[0] for i in relationship[0].leaves()])
        
        rel = " ".join(" ".join([j[0] for j in i.leaves()]) for i in relationship[1::] if i.label() != "NP")
        rel = f"{rel}::{' '.join([i[0] for i in relationship[-1].leaves()])}"
        
        if first_concept not in ontology.keys():
            ontology[first_concept] = []
        ontology[first_concept].append(rel)
        
    return ontology

sentences = [
    "The cat plays piano.",
    "Event Management is a process that monitors all events that occur through the IT infrastructure to allow for normal operation and also to detect and escalate exception conditions."
]

import re

for sentence in sentences:
    tokenised_sentence = tokenize_sentence(sentence)
    lemmatized_tokens = lemmatize_tokens(tokenised_sentence)
    tokens_with_removed_stopwords = remove_stopwords_from_tokens(lemmatized_tokens)
    noun_phrases_chunked, noun_phrases = get_noun_phrases(tokens_with_removed_stopwords)
    relationships = get_relations(noun_phrases_chunked)
    ontology = create_ontology(relationships)
        
    # print(ontology)
    for key, value in ontology.items():
        ontology[key] = [i for i in value if re.search(".+::.+", i)]
        
        # if key not in ontology_main.keys():
        #     ontology_main[key] = []            
        # print(ontology_main)
        # ontology_main[key] = [ontology_main[key].append(value) for key, value in ontology.items()]

    print(ontology)
    print("-" * 100)
    
# print(ontology_main)

{'cat': ['play::piano']}
----------------------------------------------------------------------------------------------------
{'Event Management process': ['monitor::event'], 'event': ['occur::IT infrastructure'], 'IT infrastructure': ['allow::normal operation']}
----------------------------------------------------------------------------------------------------
